In [1]:
# get the code for kaggle
!git clone https://github.com/moaaztaha/Image-Captioning
py_files_path = '/kaggle/working/Image-Captioning/'
import sys
sys.path.append(py_files_path)

fatal: destination path 'Image-Captioning' already exists and is not an empty directory.


In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import time 
import torch.backends.cudnn as cudnn
import torch.optim
import torch.utils.data
import torchvision.transforms as transforms
from torch import nn
from torch.nn.utils.rnn import pack_padded_sequence
from models import Encoder, DecoderWithAttention
from dataset import *
from utils import *
from train import *
from nltk.translate.bleu_score import corpus_bleu

In [4]:
# Model parameters
encoder_dim = 2048 # resnet101
emb_dim = 512  # dimension of word embeddings
attention_dim = 512  # dimension of attention linear layers
decoder_dim = 512  # dimension of decoder RNN
dropout = 0.5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # sets device for model and PyTorch tensors
cudnn.benchmark = True  # set to true only if inputs to model are fixed size; otherwise lot of computational overhead

# training parameters
epochs = 1  # number of epochs to train for (if early stopping is not triggered)
batch_size = 124
workers = 4
encoder_lr = 1e-4  # learning rate for encoder if fine-tuning
decoder_lr = 4e-4  # learning rate for decoder
fine_tune_encoder = False  # fine-tune encoder?
checkpoint = None  # path to checkpoint, None if none

In [5]:
DATA_NAME = 'flickr8k_5_cap_per_img_2_min_word_freq_resnet101'

# local
DATA_JSON_PATH = 'data.json'
IMGS_PATH = 'flickr/Images/'
# kaggle paths
# DATA_JSON_PATH = '/kaggle/working/Image-Captioning/data.json'
# IMGS_PATH = '../input/flickr8kimagescaptions/flickr8k/images/'

In [6]:
# load vocab
vocab = build_vocab(DATA_JSON_PATH)

100%|██████████| 40000/40000 [00:00<00:00, 386307.43it/s]


In [7]:
len(vocab)

5089

In [9]:
epochs = 1

In [10]:
t_params = {
    'data_name': DATA_NAME,
    'imgs_path': IMGS_PATH,
    'df_path': DATA_JSON_PATH,
    'vocab': vocab,
    'epochs': epochs,
    'batch_size': batch_size,
    'workers': workers,
    'decoder_lr': decoder_lr,
    'encoder_lr': encoder_lr,
    'fine_tune_encoder': fine_tune_encoder
}

m_params = {
    'attention_dim': attention_dim,
    'embed_dim': emb_dim,
    'decoder_dim': decoder_dim,
    'encoder_dim': encoder_dim,
    'dropout': dropout
}

In [11]:
t_params

{'data_name': 'flickr8k_5_cap_per_img_2_min_word_freq_resnet101',
 'imgs_path': 'flickr/Images/',
 'df_path': 'data.json',
 'vocab': <dataset.Vocabulary at 0x7fa6cd494dd0>,
 'epochs': 1,
 'batch_size': 124,
 'workers': 4,
 'decoder_lr': 0.0004,
 'encoder_lr': 0.0001,
 'fine_tune_encoder': False}

In [12]:
fit(t_params=t_params, m_params=m_params)

Loading Data
Dataset split: train
Unique images: 6000
Total size: 30000
Dataset split: val
Unique images: 1000
Total size: 5000
__________________________________________________
-------------------- Fitting --------------------
__________________________________________________
-------------------- Training --------------------
Epoch: [0][0/242]	Batch Time 12.364 (12.364)	Data Load Time 8.006 (8.006)	Loss 9.4866 (9.4866)	Top-5 Accuracy 0.000 (0.000)
Epoch: [0][100/242]	Batch Time 1.823 (1.907)	Data Load Time 0.000 (0.080)	Loss 4.9737 (5.6106)	Top-5 Accuracy 52.396 (43.264)
Epoch: [0][200/242]	Batch Time 1.813 (1.851)	Data Load Time 0.000 (0.040)	Loss 4.7632 (5.2538)	Top-5 Accuracy 54.283 (48.424)
-------------------- Validation --------------------
Validation: [0/41]	Batch Time 7.281 (7.281)	Loss 5.4750 (5.4750)	Top-5 Accuracy 51.539 (51.539)	
----- Bleu-n Scores -----
1: 55.733232557025005
2: 33.00266069909076
3: 17.830606478271484
4: 9.696190714028482
-------------------------

 * L